Projeto: BETA BANK
Os clientes do Beta Bank estão saindo: pouco a pouco, escapulindo todo mês. Os banqueiros descobriram que é mais barato manter os clientes existentes do que atrair novos. Precisamos prever se um cliente vai deixar o banco em breve.

1  Importando as bibliotecas e os dados

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, confusion_matrix, recall_score, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from sklearn.tree import DecisionTreeRegressor

In [ ]:
data = pd.read_csv('/datasets/Churn.csv')

2  Analizando os dados¶

In [ ]:
data.info()
print(data.sample(5))

Aparentemente os dados possuem tipo de acordo com o previsto. Entretanto a coluna 'Tenure'apresenta cerca de 9% de valores ausentes. As estatísticas descritivas serão analisadas para perceber qual a melhor maneira de realizar o preenchimento destes valores ausentes. 

As colunas 'Gender' e 'Geography' estão em formato óbject, mas para a posterior análise, será necessário passar para o formato de variáveis dummy, através da aplicação da codificação one-hot a variável 'Gender'e a variável 'Geography' será transformada em ordinal. Isso porque, o gênero e o local da conta podem ser fatores que expliquem a saída de usuários do banco.

As colunas 'HasCrCard', 'IsActiveMember' e 'Exited' já estão no formato int64 com valores binários (0 e 1), elas já são consideradas variáveis dummy. Não é necessário aplicar a codificação one-hot nessas colunas, pois elas já estão no formato apropriado para modelos de machine learning.

As colunas 'rownumber' (índice das strings de dados), customerId (identificador exclusivo do cliente) e surname (sobrenome) serão excluídas da base de dados, por não serem possíveis variáveis explicativas para a saída de usuários do banco.

3  Preparando os dados
3.1  Transformando a coluna Gender em variável dummy

In [ ]:
print(data['Gender'].unique()) # Visualizando os valores únicos na coluna 'Gender'
# Transformando a coluna 'Gender'em Codificação One-Hot (OHE) 
# Retirando uma das colunas para não haver multicolinearidade entre elas
data = pd.get_dummies(data, columns=['Gender'], drop_first=True) 



3.2  Transformando a coluna Geography em ordinal

In [ ]:
print(data['Geography'].unique()) # Visualizando os valores únicos na coluna 'Geography'

# Criando uma instância da classe OrdinalEncoder
encoder = OrdinalEncoder()

# Ajustando o encoder aos dados de 'Geography'
encoder.fit(data[['Geography']])  # Passando apenas a coluna 'Geography'

# Transformando 'Geography' em uma representação ordinal
data['Geography_ordinal'] = encoder.transform(data[['Geography']])

# Removendo a coluna original 'Geography'
data.drop(columns=['Geography'], inplace=True)

3.3  Descartando as colunas 'RowNumber', 'CustomerId' e 'Surname'

In [ ]:
data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

3.4  Divisão dos dados em conjuntos de treinamento, validação e teste

Dividir os dados em conjuntos de treinamento, teste e validação é essencial para avaliar adequadamente o desempenho do modelo de machine learning. O conjunto de treinamento é usado para treinar o modelo, o conjunto de teste é usado para avaliar o desempenho do modelo em dados não vistos e o conjunto de validação é usado para ajustar os hiperparâmetros do modelo e evitar o superajuste. Essa divisão garante que o modelo seja avaliado de forma justa e imparcial, fornecendo uma estimativa confiável de seu desempenho em dados reais.

In [ ]:
# definindo features e target
features = data.drop(columns=['Exited'])
target = data['Exited']


# Divisão dos dados
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=42)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=42)

3.5  Valores ausentes na coluna 'Tenure'

Em primeiro lugar, uma análise das estatísticas descritivas para a coluna 'Tenure' será realizada. Bem como uma análise gráfica para verificar a distribuição da variável.

In [ ]:
data['Tenure'].describe()

# Plotando o histograma
plt.hist(data['Tenure'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribuição de Tenure')
plt.xlabel('Tenure')
plt.ylabel('Frequência')
plt.show()


# Plotando gráfico de densidade
sns.displot(data['Tenure'], kde=True, color='blue')
plt.title('Distribuição de Tenure')
plt.xlabel('Tenure')
plt.ylabel('Densidade')
plt.show()

Aparentemente, a distribuição não é normalmente distribuída. Sendo assim, abaixo é realizado um teste de normalidade de Shapiro. 

In [ ]:
data_to_test = data['Tenure'].sample(5000)

# Removendo os valores ausentes da coluna 'Tenure'
data_to_test = data_to_test.dropna()

# Teste de normalidade de Shapiro-Wilk
stat, p_value = shapiro(data_to_test)

# Imprimindo o resultado do teste
print("Estatística de teste:", stat)
print("Valor p:", p_value)

# Interpretando o resultado do teste
alpha = 0.05  # Nível de significância
if p_value > alpha:
    print("Os dados parecem seguir uma distribuição normal (não podemos rejeitar a hipótese nula)")
else:
    print("Os dados não seguem uma distribuição normal (rejeitamos a hipótese nula)")

Como a variável Tenure não segue distribuição normal, para a imputação de valores ausentes foi escolhido o método pelo modelo de árvore de decisão.

3.5.1  Imputando valores nos dados de treinamento¶

In [ ]:
# Criando um modelo de árvore de decisão
tree_model = DecisionTreeRegressor(random_state=42)

# Removendo os valores ausentes da coluna 'Tenure' nos dados de treinamento
features_train_no_missing = features_train.dropna(subset=['Tenure'])
target_train_no_missing = target_train[features_train['Tenure'].notna()]

# Separando as features e o target sem valores ausentes
X_train = features_train_no_missing.drop('Tenure', axis=1)
y_train = target_train_no_missing

# Treinando o modelo de árvore de decisão
tree_model.fit(X_train, y_train)

# Identificando os índices dos valores ausentes nos dados de treinamento
missing_indices = features_train[features_train['Tenure'].isnull()].index

# Preenchendo os valores ausentes usando o modelo treinado
X_missing = features_train.loc[missing_indices].drop('Tenure', axis=1)
imputed_values = tree_model.predict(X_missing)

# Criando uma cópia dos dados de treinamento para evitar o aviso que apareceu anteriormente
features_train_copy = features_train.copy()

# Preenchendo os valores ausentes nos dados de treinamento
features_train_copy.loc[missing_indices, 'Tenure'] = imputed_values

# Atribuindo ao DataFrame original
features_train = features_train_copy

# Verificando se ainda existem valores ausentes nos dados de treinamento
print(features_train['Tenure'].isnull().sum())

Os valores ausentes da coluna 'Tenure' foram preenchidos com sucesso pelo modelo de árvore de decisão.

3.5.2  Imputando valores nos dados de validação

In [ ]:
# Remover os valores ausentes dos dados de validação
features_valid_no_missing = features_valid.dropna(subset=['Tenure'])

# Usar o modelo treinado para prever os valores ausentes nos dados de validação
X_valid_missing = features_valid.loc[features_valid.index.difference(features_valid_no_missing.index)].drop('Tenure', axis=1)
imputed_values_valid = tree_model.predict(X_valid_missing)

# Criando uma cópia dos dados de validação para evitar o aviso alerta de cópia
features_valid_copy = features_valid.copy()

# Preencher os valores ausentes nos dados de validação
features_valid_copy.loc[X_valid_missing.index, 'Tenure'] = imputed_values_valid

# Atribuindo ao DataFrame original
features_valid = features_valid_copy


3.5.3  Imputando valores nos dados de teste

In [ ]:
# Remover os valores ausentes dos dados de teste
features_test_no_missing = features_test.dropna(subset=['Tenure'])

# Usar o modelo treinado para prever os valores ausentes nos dados de teste
X_test_missing = features_test.loc[features_test.index.difference(features_test_no_missing.index)].drop('Tenure', axis=1)
imputed_values_test = tree_model.predict(X_test_missing)

# Criando uma cópia dos dados de teste para evitar o aviso alerta de cópia
features_test_copy = features_test.copy()

# Preencher os valores ausentes nos dados de teste
features_test_copy.loc[X_test_missing.index, 'Tenure'] = imputed_values_test

# Atribuindo ao DataFrame original
features_test = features_test_copy

3.6  Padronizando as características numéricas


A padronização das características numéricas é importante porque muitos algoritmos de machine learning assumem que todas as características têm a mesma escala. A padronização transforma as características para que elas tenham média zero e desvio padrão unitário, o que ajuda a evitar que características com magnitudes muito diferentes dominem o modelo. Isso melhora a estabilidade e a eficiência dos algoritmos de aprendizado, garantindo que o modelo seja treinado de maneira mais consistente e robusta.

In [ ]:
# Padronizando as características numéricas
scaler = StandardScaler()
scaler.fit(features_train)  # Ajuste do scaler aos dados de treinamento
features_train_scaled = scaler.transform(features_train)  # Padronizando as características de treinamento
features_valid_scaled = scaler.transform(features_valid)  # Padronizando as características de validação
features_test_scaled = scaler.transform(features_test)  # Padronizando as características de teste, se necessário


4  Treinando o modelo

4.1  Exame do equilíbrio de classes

In [ ]:
class_distribution = target.value_counts(normalize=True) # Examinando o equilíbrio das classes

print("Distribuição das classes:")
print(class_distribution)

Isso significa que a classe 0 (por exemplo, clientes que não deixaram o serviço) representa aproximadamente 79.63% do conjunto de dados, enquanto a classe 1 (clientes que deixaram o serviço) representa aproximadamente 20.37%. Essa é uma distribuição desbalanceada, onde uma classe é significativamente mais prevalente do que a outra.

4.2  Treino do modelo sem levar em conta o desequilíbrio

In [ ]:
# Treinando o modelo inicial
model = RandomForestClassifier(random_state=42) #LogisticRegression(random_state=42)
model.fit(features_train, target_train)

# Fazendo previsões nos dados de validação
predictions = model.predict(features_valid)

# Avaliando o desempenho do modelo
accuracy = accuracy_score(target_valid, predictions)
precision = precision_score(target_valid, predictions)
f1 = f1_score(target_valid, predictions)
auc_roc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])
conf_matrix = confusion_matrix(target_valid, predictions)

# Imprimindo as métricas de desempenho
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("Matriz de Confusão:")
print(conf_matrix)

O modelo RandomForestClassifier foi treinado sem levar em conta o desequilíbrio inicial das classes. Os resultados mostram que o modelo possui um desempenho razoável na previsão da saída dos clientes do banco. Com uma acurácia de 0.8635, o modelo classifica corretamente aproximadamente 86.1% das instâncias. A precisão de 0.7734 indica que, das instâncias previstas como saída, cerca de 77.3% são realmente saídas, enquanto o F1-score de 0.5919 é uma média harmônica entre precisão e recall. A área sob a curva ROC (AUC-ROC) de 0.8511 indica que o modelo tem uma boa capacidade de distinguir entre as classes positiva e negativa. A matriz de confusão mostra que o modelo está capturando uma quantidade significativa de verdadeiros positivos, mas também está classificando algumas instâncias negativas erroneamente como positivas. Isso sugere que ainda há espaço para melhorias, especialmente na redução dos falsos positivos e na melhoria do recall.

5  Melhorando a qualidade do modelo

5.1  Ajuste de ponderação de classe

In [ ]:
# Criando e treinando o modelo com ajuste de ponderação da classe
model = RandomForestClassifier(class_weight='balanced', random_state=42)# LogisticRegression(class_weight='balanced', random_state=42)
model.fit(features_train, target_train)

# Fazendo previsões
predictions = model.predict(features_valid)

# Calcular métricas
accuracy = accuracy_score(target_valid, predictions)
precision = precision_score(target_valid, predictions)
f1 = f1_score(target_valid, predictions)
auc_roc = roc_auc_score(target_valid, predictions)
conf_matrix = confusion_matrix(target_valid, predictions)

# Imprimindo as métricas de desempenho
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("Matriz de Confusão:")
print(conf_matrix)

Ao ajustar a ponderação das classes para lidar com o desequilíbrio, observamos que a acurácia, precisão e F1-score permaneceram praticamente inalteradas em relação ao modelo anterior, indicando que o ajuste de ponderação não teve um impacto significativo nessas métricas. No entanto, a AUC-ROC diminuiu ligeiramente para 0.7139, sugerindo uma capacidade ligeiramente inferior do modelo em distinguir entre as classes positiva e negativa em comparação com o modelo anterior. A matriz de confusão permaneceu semelhante, mostrando que o modelo ainda está classificando algumas instâncias negativas erroneamente como positivas. Isso sugere que o ajuste de ponderação pode não ser a melhor abordagem para melhorar o desempenho do modelo neste caso específico.

5.2  Superamostragem

In [ ]:
# Função para superamostragem (upsampling)
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=42
    )

    return features_upsampled, target_upsampled

# Aplicando superamostragem nos dados de treinamento
features_upsampled, target_upsampled = upsample(
    features_train, target_train, 10
)

# Treinando o modelo de Regressão Logística com superamostragem
log_reg = RandomForestClassifier(random_state=42) #LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(features_upsampled, target_upsampled)

# Fazendo previsões no conjunto de validação
predicted_valid = log_reg.predict(features_valid)

# Calculando e imprimindo a métrica F1
print('F1:', f1_score(target_valid, predicted_valid))

Ao aplicar a técnica de superamostragem (upsampling) nos dados de treinamento, aumentando a quantidade de instâncias da classe minoritária, observamos que o modelo RandomForestClassifier treinado apresentou um F1-score de aproximadamente 0.589 no conjunto de validação. Isso representa uma melhoria em relação ao modelo anterior que não considerou a superamostragem, indicando que essa técnica ajudou o modelo a melhorar sua capacidade de prever corretamente as instâncias da classe positiva. No entanto, é importante notar que o resultado ainda pode ser aprimorado por meio de outras técnicas de ajuste de modelo.

5.3  Subamostragem

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=42)]
        + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=42)]
        + [target_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=42
    )

    return features_downsampled, target_downsampled


features_downsampled, target_downsampled = downsample(
    features_train, target_train, 0.1
)

model = RandomForestClassifier(random_state=42) #LogisticRegression(solver='liblinear', random_state=42)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))

A utilização da técnica de subamostragem (undersampling) resultou em um F1-score de 0.48, o que é menor em comparação com a superamostragem. Isso pode indicar que a subamostragem pode ter reduzido a quantidade de dados disponíveis para o modelo aprender, o que pode ter impactado negativamente na capacidade de generalização do modelo.

5.4  Ajuste de limiar

In [ ]:
model = RandomForestClassifier(random_state=42) #LogisticRegression(random_state=42, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.1, 0.4, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    
    print(
        'Limiar = {:.2f} | Precisão = {:.3f}, Sensibilidade = {:.3f}'.format(
            threshold, precision, recall
        ))

5.5  Melhor limiar

In [ ]:
# Probabilidades previstas pelo modelo
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

# Defina uma faixa de limiares
thresholds = np.arange(0, 1, 0.02)

best_f1 = 0
best_threshold = 0

# Itere sobre os limiares e calcule o F1-score para cada um
for threshold in thresholds:
    predicted_valid = probabilities_one_valid > threshold
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)
    f1 = f1_score(target_valid, predicted_valid)
    
    # Atualize o melhor F1-score e o melhor limiar
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print("Limiar ideal:", best_threshold)
print("Melhor F1-score:", best_f1)

O ajuste de limiar é uma etapa importante para maximizar o desempenho do modelo, especialmente quando se trata de problemas de classificação desbalanceados, como no seu caso de prever a saída de clientes de um banco. No entanto, ao analisar os resultados, percebemos que o melhor limiar encontrado foi de 0.38, com um F1-score de 0.6157. Isso significa que esse limiar produz um equilíbrio entre precisão e recall, resultando em um bom desempenho geral do modelo.

Ao ajustar o limiar para diferentes valores na faixa de 0.1 a 0.4, observamos que o F1-score aumenta até atingir o pico em 0.3 e, em seguida, começa a diminuir gradualmente. Isso indica que o limiar de 0.3 é o ponto ideal que maximiza o F1-score para o seu problema específico.

6  Teste final

In [ ]:
# Testando diferentes combinações de hiperparâmetros
hyperparameters = [
    {'n_estimators': 100, 'max_depth': None, 'class_weight': None},
    {'n_estimators': 100, 'max_depth': 5, 'class_weight': None},
    {'n_estimators': 100, 'max_depth': 10, 'class_weight': None},
    {'n_estimators': 100, 'max_depth': None, 'class_weight': 'balanced'},
    {'n_estimators': 100, 'max_depth': 5, 'class_weight': 'balanced'},
    {'n_estimators': 100, 'max_depth': 10, 'class_weight': 'balanced'},
    ]

best_f1 = 0
best_model = None

# Iterando sobre as diferentes combinações de hiperparâmetros
for params in hyperparameters:
    # Criando e treinando o modelo com a combinação atual de hiperparâmetros
    model = RandomForestClassifier(random_state=42, **params)
    model.fit(features_train, target_train)
    
    # Fazendo previsões nos dados de validação
    predictions_valid = model.predict(features_valid)
    
    # Calculando o F1-score
    f1 = f1_score(target_valid, predictions_valid)
    
    # Atualizando o melhor F1-score e o melhor modelo
    if f1 > best_f1:
        best_f1 = f1
        best_model = model

# Avaliando o desempenho do melhor modelo nos dados de teste
predictions_test = best_model.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test)
precision = precision_score(target_test, predictions_test)
recall = recall_score(target_test, predictions_test)
f1 = f1_score(target_test, predictions_test)
auc_roc = roc_auc_score(target_test, best_model.predict_proba(features_test)[:, 1])

# Imprimindo as métricas do modelo
print("Desempenho final do melhor modelo RandomForestClassifier no conjunto de teste:")
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)


Ao testar diferentes combinações de hiperparâmetros para o modelo RandomForestClassifier, observamos que a melhor combinação resultou em um F1-score de aproximadamente 0.602 no conjunto de teste. Isso significa que o modelo é capaz de prever corretamente cerca de 60,2% das instâncias positivas, alcançando uma acurácia de 0.8385. A precisão do modelo é de aproximadamente 0.584, o que indica que cerca de 58,4% das instâncias previstas como positivas são realmente positivas. O recall, que mede a proporção de instâncias positivas que foram corretamente identificadas pelo modelo, é de aproximadamente 0.621. Além disso, a área sob a curva ROC (AUC-ROC) é de 0.855, indicando uma boa capacidade do modelo em distinguir entre as classes positiva e negativa. Esses resultados sugerem que o modelo RandomForestClassifier ajustado com a melhor combinação de hiperparâmetros tem um desempenho satisfatório na previsão da saída de clientes do banco.

7  Conclusões

O projeto de previsão de saída de clientes do banco foi conduzido com o objetivo de desenvolver um modelo capaz de identificar clientes propensos a encerrar sua relação com o banco. Inicialmente, exploramos e pré-processamos os dados, lidando com valores ausentes, codificação de variáveis categóricas e divisão dos dados em conjuntos de treinamento, validação e teste. Em seguida, treinamos vários modelos de classificação, como RandomForestClassifier e LogisticRegression, utilizando diferentes abordagens para lidar com o desequilíbrio de classes, como ajuste de ponderação, superamostragem e seleção de hiperparâmetros.

Após avaliar o desempenho de cada modelo com métricas como acurácia, precisão, recall, F1-score e AUC-ROC, identificamos o RandomForestClassifier ajustado com a melhor combinação de hiperparâmetros como nosso modelo final. Este modelo obteve um F1-score de aproximadamente 0.602 no conjunto de teste, o que indica uma capacidade razoável de prever corretamente os clientes que estão propensos a sair do banco. No entanto, ainda há espaço para melhorias, especialmente na precisão e recall do modelo.

Em resumo, o projeto demonstrou que é possível construir um modelo de previsão de saída de clientes do banco com base em dados históricos. A implementação bem-sucedida deste modelo pode fornecer insights valiosos para a gestão de relacionamento com os clientes, permitindo a adoção de estratégias proativas para retenção de clientes e mitigação do churn, contribuindo assim para a sustentabilidade e crescimento do banco.